In [1]:
import os
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
import pandas as pd
import pprint
import datetime as dt

In [2]:
url_corona = "https://www.worldometers.info/coronavirus/#countries"
html_corona = requests.get(url_corona)
html_corona = html_corona.text
#html_corona

In [3]:
soup_corona = bs(html_corona, 'html.parser')

In [4]:
corona_table = soup_corona.find(class_="table-bordered")

corona_table = str(corona_table)


In [5]:
corona_df = pd.read_html(corona_table)[0]

#replacing blanks with 0
corona_df.fillna(0,inplace = True)

#removing the total row)
corona_df = corona_df[~corona_df['Country,Other'].isin(['Total:', 'Total', 'total'])]
corona_df.to_csv('new_data.csv')
#replace items using wild card and regex to have that applied
corona_df["NewCases"].replace("\,","",regex = True, inplace = True)
corona_df["NewCases"].replace("\+","", regex = True, inplace = True)
corona_df["NewCases"] = corona_df["NewCases"].astype("float")
corona_df["Date"]= dt.datetime.today()

In [6]:
# storing the rate of growth for analysis

#importing previous data
growth_last = pd.read_csv("growth.csv")
# #staging new data to add
growth_df = pd.DataFrame()


growth_df["Date"]=[0]
growth_df["NewCases"]=[0]
growth_df["TotalCases"] =[0]
growth_df["VarianceMultiple"]=[0]
growth_df["GrowthFactor"]=[0]

growth_df["Date"]= dt.datetime.today()
growth_df["NewCases"] = corona_df["NewCases"].sum()
growth_df["TotalCases"] =corona_df["TotalCases"].sum()
growth_df["VarianceMultiple"]= 1 + (growth_df["NewCases"]/growth_df["TotalCases"])
growth_df["GrowthFactor"]=[growth_df["NewCases"][0]/growth_last["NewCases"][len(growth_last)-1]]

#appending new data
    
updated_growth_df = growth_last.append(growth_df)
updated_growth_df.reset_index(inplace=True,drop=True)
updated_growth_df.to_csv('growth.csv',index = False)
updated_growth_df.to_html('growth.html',index = False)
updated_growth_df.to_json('growth.json',orient = "records")
updated_growth_df

,Date,NewCases,TotalCases,VarianceMultiple,GrowthFactor
0,2020-03-11 00:57:05.757569,297.0,119220,1.002491,1.000000
1,2020-03-11 01:03:51.872781,297.0,119220,1.002491,1.000000
2,2020-03-11 01:16:11.835765,302.0,119225,1.002533,1.016835
3,2020-03-11 20:18:49.306369,7113.0,126061,1.056425,23.552980
4,2020-03-11 20:38:41.539226,7113.0,126061,1.056425,1.000000
5,2020-03-12 00:06:55.727048,178.0,126367,1.001409,0.025025
6,2020-03-13 19:58:32.715439,10783.0,145360,1.074181,60.578652
7,2020-03-13 20:00:07.275754,10783.0,145360,1.074181,1.000000
8,2020-03-13 21:43:27.476069,11051.0,145628,1.075885,1.024854
9,2020-03-14 01:31:08.811147,201.0,145673,1.001380,0.018188


In [7]:
# adding lat long info 
countries_df = pd.read_csv("country_lan_lng.csv",engine='python')
merged = corona_df.merge(countries_df, left_on='Country,Other', right_on='name')
df_used = merged.drop(countries_df.columns[0:-2], axis=1)

df_used.rename(columns={"Country,Other":"Country","Serious,Critical": "CriticalCases"}, inplace = True)
#replace items using wild card and regex to have that applied
df_used["NewCases"].replace("\,","",regex = True, inplace = True)
df_used["NewCases"].replace("\+","", regex = True, inplace = True)

#add time stamp


df_country_list = df_used["Country"].tolist()
df_used
#df_country_list

,Country,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,ActiveCases,CriticalCases,Tot Cases/1M pop,Date,Longitude,Latitude
0,China,80824,11.0,3189.0,13.0,65573.0,12062,3610.0,56.2,2020-03-14 10:17:29.833934,103.819074,36.561765
1,Italy,17660,0.0,1266.0,0.0,1439.0,14955,1328.0,292.1,2020-03-14 10:17:29.833934,12.070013,42.796626
2,Iran,12729,1365.0,611.0,97.0,4339.0,7779,0.0,151.5,2020-03-14 10:17:29.833934,54.274070,32.575033
3,S. Korea,8086,107.0,72.0,5.0,714.0,7300,59.0,157.7,2020-03-14 10:17:29.833934,127.839161,36.385240
4,Spain,6023,791.0,191.0,58.0,517.0,5315,272.0,128.8,2020-03-14 10:17:29.833934,-3.647550,40.244487
...,...,...,...,...,...,...,...,...,...,...,...,...
125,Mongolia,1,0.0,0.0,0.0,0.0,1,0.0,0.3,2020-03-14 10:17:29.833934,103.052998,46.826815
126,Rwanda,1,1.0,0.0,0.0,0.0,1,0.0,0.1,2020-03-14 10:17:29.833934,29.919885,-1.990338
127,Saint Lucia,1,0.0,0.0,0.0,0.0,1,0.0,0.0,2020-03-14 10:17:29.833934,-60.969699,13.894795
128,Suriname,1,0.0,0.0,0.0,0.0,1,0.0,0.0,2020-03-14 10:17:29.833934,-55.912346,4.130554


In [8]:
df_used.to_csv('data.csv')
df_used.to_html('data.html')

In [9]:
corona_json = df_used.to_json(orient='records')
df_used.to_json('data.json',orient='records')

In [10]:
corona_json

'[{"Country":"China","TotalCases":80824,"NewCases":11.0,"TotalDeaths":3189.0,"NewDeaths":13.0,"TotalRecovered":65573.0,"ActiveCases":12062,"CriticalCases":3610.0,"Tot\\u00a0Cases\\/1M pop":56.2,"Date":1584181049833,"Longitude":103.8190735,"Latitude":36.56176546},{"Country":"Italy","TotalCases":17660,"NewCases":0.0,"TotalDeaths":1266.0,"NewDeaths":0.0,"TotalRecovered":1439.0,"ActiveCases":14955,"CriticalCases":1328.0,"Tot\\u00a0Cases\\/1M pop":292.1,"Date":1584181049833,"Longitude":12.07001339,"Latitude":42.79662641},{"Country":"Iran","TotalCases":12729,"NewCases":1365.0,"TotalDeaths":611.0,"NewDeaths":97.0,"TotalRecovered":4339.0,"ActiveCases":7779,"CriticalCases":0.0,"Tot\\u00a0Cases\\/1M pop":151.5,"Date":1584181049833,"Longitude":54.27407004,"Latitude":32.57503292},{"Country":"S. Korea","TotalCases":8086,"NewCases":107.0,"TotalDeaths":72.0,"NewDeaths":5.0,"TotalRecovered":714.0,"ActiveCases":7300,"CriticalCases":59.0,"Tot\\u00a0Cases\\/1M pop":157.7,"Date":1584181049833,"Longitude":